---
title: Data Lake
author: Jesús López M.
abstract: En este apartado mostramos la creación del Data Lake.
date: 2025-07-22
date-format: MMM, YYYY
date-modified: last-modified
toc: true
toc-location: left
lan: es
format:
  html
jupyter: python3
---

## Environment settings

In [1]:
# import libraries
import numpy as np
import pandas as pd
import geopandas as gpd
import polars as pl
import duckdb as db
from pathlib import Path
from shapely.wkb import loads

## Layers

In [3]:
# define base folder
base = Path('../')

In [4]:
# create bronze folder (raw)
bronze_folder = base/'1-bronze'
bronze_folder.mkdir(parents=True, exist_ok=True)

In [5]:
# create silver folder (cleaned parquet)
silver_folder = base/'2-silver'
silver_folder.mkdir(parents=True, exist_ok=True)

In [6]:
# create golden folder (aggregate)
golden_folder = base/'3-golden'
golden_folder.mkdir(parents=True, exist_ok=True)

## ETL

### Step 1. Locate Bronze and Silver File Paths

In [7]:
# name bronze files
# json
municipios_json = bronze_folder/'mexico.json'
estados_json = bronze_folder/'estados.geojson'

# excels
edo_fza_pol_excel = bronze_folder/'edo_fza_pol.xlsx'
edo_fza_pol_mun_excel = bronze_folder/'edo_fza_pol_mun.xlsx'
edo_fza_pols_excel = bronze_folder/'edo_fza_pols_201805_202506.xlsx'
pob_estados_excel = bronze_folder/'pob_entidades_2024-09.xlsx'
fasp_asignaciones_excel = bronze_folder/'fasp_asignaciones_20079_2025.xlsx'
dgp_staff_excel = bronze_folder/'dgp_staff.xlsx'
areas_fiscalias_2024_excel = bronze_folder/'areas_fiscalias_2024.xlsx'
areas_policias_2024_excel = bronze_folder/'areas_policias_2024.xlsx'
conasami_pol_est_excel = bronze_folder/'conasami_pol_est_2024_2025.xlsx'
fasp_2024_concentrado_csv = bronze_folder/'fasp_2024_concentrado.csv'
fasp_2025_concentrado_excel = bronze_folder/'fasp_2025_concentrado.xlsx'
fasp_asignaciones_excel = bronze_folder/'fasp_asignaciones_2007_2025.xlsx'
pob_coord_localidades_excel = bronze_folder/'pob_localidades_coordenadas_2025-05.xlsx'
encuesta_fasp_spss_csv = bronze_folder/'data.csv'

In [8]:
# name silver files
municipios_geojson = silver_folder/'municipios_geojson.parquet'
estados_geojson = silver_folder/'estados_geojson.parquet'
edo_fza_pol = silver_folder/'edo_fza_pol.parquet'
edo_fza_pol_mun = silver_folder/'edo_fza_pol_mun.parquet'
edo_fza_pols = silver_folder/'edo_fza_pols.parquet'
pob_estados = silver_folder/'pob_estados.parquet'
fasp_asignaciones = silver_folder/'fasp_asignaciones.parquet'
dgp_staff = silver_folder/'dgp_staff.parquet'
areas_fiscalias_2024 = silver_folder/'areas_fiscalias_2024.parquet'
areas_policias_2024 = silver_folder/'areas_policias_2024.parquet'
conasami_pol_est = silver_folder/'conasami_pol_est_2024_2025.parquet'
fasp_2024_concentrado = silver_folder/'fasp_2024_concentrado.parquet'
fasp_2025_concentrado = silver_folder/'fasp_2025_concentrado.parquet'
fasp_asignaciones = silver_folder/'fasp_asignaciones_2007_2025.parquet'
pob_coord_localidades = silver_folder/'pob_coord_localidades.parquet'
encuesta_fasp_spss = silver_folder/'encuesta_fasp_spss.parquet'

### Step 2. Transform and Load to Silver Layer

#### Geospatial files

In [10]:
def convert_geoparquet(origin_file, parquet_file):
    # extract geojson with geopandas
    gdf = gpd.read_file(origin_file)
    # save to parquet file from geopandas
    gdf.to_parquet(parquet_file)

In [11]:
convert_geoparquet(municipios_json, municipios_geojson)

In [12]:
convert_geoparquet(estados_json, estados_geojson)

#### Flat files

In [ ]:
# areas fiscalias 2024
(
    pl.read_excel(areas_fiscalias_2024_excel)
        .write_parquet(areas_fiscalias_2024)
)

In [33]:
# areas policias 2024
(
    pl.read_excel(areas_policias_2024_excel)
        .write_parquet(areas_policias_2024)
)

In [38]:
# conasami policia estatal 2024-2025
(
    pl.read_excel(conasami_pol_est_excel)
        .write_parquet(conasami_pol_est)
)

In [ ]:
# dgp staff
(
    pl.read_excel(dgp_staff_excel)
        .with_columns(
            pl.col('Cumpleaños').dt.strftime("%B, %d").alias('Cumple')
            )
        .write_parquet(dgp_staff)
)

In [42]:
# fasp asignaciones anuales 2007-2025
(
    pl.read_excel(fasp_asignaciones_excel)
        .write_parquet(fasp_asignaciones)        
)

In [55]:
# fasp concentrado 2024
(
    pl.read_excel(fasp_2024_concentrado_csv)
        .write_parquet(fasp_2024_concentrado)
)

In [56]:
# fasp concentrado 2025
(
    pl.read_excel(fasp_2025_concentrado_excel)
        .write_parquet(fasp_2025_concentrado)
)

In [61]:
# pob localidades coordenadas may 2025
(
    pl.read_excel(pob_coord_localidades_excel)
        .write_parquet(pob_coord_localidades)
)

In [10]:
# encuesta fasp 2024 spss
(
    pl.read_csv(encuesta_fasp_spss_csv)
        .write_parquet(encuesta_fasp_spss)
)

### Step 3. Create Aggregated Tables in golden Layer

### Step 4. Query the data lake using DuckDB or Polars

### Create geopandas dataframe from duckdb geotable

In [ ]:
def export_geodataframe(table, parquet_name):
    # export duckdb geotable to geoparquet
    conn.execute(f''' 
        COPY '{table}' TO '{parquet_name}' (FORMAT parquet);
    ''')

In [ ]:
export_geodataframe(estados_geojson, 'estados.parquet')

In [ ]:
def read_geopandas(parquet_name):
    # convert geoparquet to geopandas dataframe
    # Step 1: Read the GeoParquet file into a pandas DataFrame
    df = pd.read_parquet(parquet_name)

    # Step 2: Convert the geometry column from WKB format
    # The 'loads' function from shapely.wkb handles this conversion
    # We apply this function to every row in the 'geometry' column
    geometry_series = gpd.GeoSeries(df['geometry'].apply(loads))

    # Step 3: Create the GeoDataFrame
    # We pass the original dataframe and the converted geometry series
    gdf = gpd.GeoDataFrame(df, geometry=geometry_series)

    # Optional but recommended: set the Coordinate Reference System (CRS)
    gdf = gdf.set_crs(epsg=4326, inplace=True)

    return gdf

In [ ]:
gdf_estados = read_geopandas('estados.parquet')
gdf_estados.head(2)